In [38]:
#Import Dependencies#
import os
import pandas as pd
from glob import glob
import requests
import pandas as pd
import geojson

In [39]:
#Using glob module to store csv rows#
filenames = glob('new_york/*.csv')
dataframe_list = [pd.read_csv(f) for f in filenames]
dataframe_list.append(pd.read_csv("new_york_current.csv", low_memory=False))

In [40]:
#Concatenate all stored dataframes created above#
all_data_df = pd.concat(dataframe_list, sort=False)

In [41]:
#Display Data#
all_data_df.head()

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,...,latitude,longitude,last_modified,survey_id,country,city,bathrooms,name,property_type,location
0,105.0,500.0,Private room,Manhattan,Hell's Kitchen,39.0,5.0,7.0,1.0,174.0,...,40.762099,-73.996022,2014-12-03 12:58:59.578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2515.0,2758.0,Private room,Manhattan,Harlem,106.0,4.5,2.0,1.0,69.0,...,40.799205,-73.953676,2014-12-04 02:53:51.833,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2684.0,2965.0,Private room,Manhattan,Upper West Side,3.0,5.0,2.0,1.0,152.0,...,40.792353,-73.967807,2014-12-03 19:36:20.383,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2993.0,3352.0,Entire home/apt,Manhattan,Lower East Side,2.0,4.5,2.0,1.0,175.0,...,40.723593,-73.991188,2014-12-04 00:08:21.539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3330.0,4177.0,Private room,Brooklyn,Williamsburg,11.0,5.0,2.0,1.0,152.0,...,40.708558,-73.942362,2014-12-03 04:18:01.520,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
#List Column Names#
list(all_data_df.columns.values)

['room_id',
 'host_id',
 'room_type',
 'borough',
 'neighborhood',
 'reviews',
 'overall_satisfaction',
 'accommodates',
 'bedrooms',
 'price',
 'minstay',
 'latitude',
 'longitude',
 'last_modified',
 'survey_id',
 'country',
 'city',
 'bathrooms',
 'name',
 'property_type',
 'location']

In [43]:
#Keeping only the relevant columns#
all_data_df = all_data_df.loc[:,["last_modified", "price", "neighborhood", "borough", "latitude", "longitude", "overall_satisfaction", "room_type"]]
all_data_df.head()

,last_modified,price,neighborhood,borough,latitude,longitude,overall_satisfaction,room_type
0,2014-12-03 12:58:59.578,174.0,Hell's Kitchen,Manhattan,40.762099,-73.996022,5.0,Private room
1,2014-12-04 02:53:51.833,69.0,Harlem,Manhattan,40.799205,-73.953676,4.5,Private room
2,2014-12-03 19:36:20.383,152.0,Upper West Side,Manhattan,40.792353,-73.967807,5.0,Private room
3,2014-12-04 00:08:21.539,175.0,Lower East Side,Manhattan,40.723593,-73.991188,4.5,Entire home/apt
4,2014-12-03 04:18:01.520,152.0,Williamsburg,Brooklyn,40.708558,-73.942362,5.0,Private room


In [44]:
#Cleaning Up Column Names#
all_data_df = all_data_df.rename(index=str, columns={"last_modified": "Date", "overall_satisfaction": "Review", "room_type": "Room Type"})
all_data_df = all_data_df.rename(str.title, axis='columns')
all_data_df

,Date,Price,Neighborhood,Borough,Latitude,Longitude,Review,Room Type
0,2014-12-03 12:58:59.578,174.0,Hell's Kitchen,Manhattan,40.762099,-73.996022,5.0,Private room
1,2014-12-04 02:53:51.833,69.0,Harlem,Manhattan,40.799205,-73.953676,4.5,Private room
2,2014-12-03 19:36:20.383,152.0,Upper West Side,Manhattan,40.792353,-73.967807,5.0,Private room
3,2014-12-04 00:08:21.539,175.0,Lower East Side,Manhattan,40.723593,-73.991188,4.5,Entire home/apt
4,2014-12-03 04:18:01.520,152.0,Williamsburg,Brooklyn,40.708558,-73.942362,5.0,Private room
5,2014-12-03 12:47:52.616,104.0,Bedford-Stuyvesant,Brooklyn,40.678300,-73.944627,4.5,Entire home/apt
6,2014-12-04 05:49:27.529,117.0,Clinton Hill,Brooklyn,40.685138,-73.959757,4.5,Private room
7,2014-12-04 03:42:36.846,211.0,Crown Heights,Brooklyn,40.674376,-73.933990,5.0,Entire home/apt
8,2014-12-03 04:22:35.240,65.0,Harlem,Manhattan,40.824861,-73.949982,4.5,Private room
9,2014-12-03 19:41:24.512,234.0,Hell's Kitchen,Manhattan,40.762600,-73.993045,4.5,Entire home/apt


In [45]:
#Lookup all null value types and drop extraneous rows#
null_columns=all_data_df.columns[all_data_df.isnull().any()]
print(all_data_df[null_columns].isnull().sum())
num_rows= 1186590
num_rows_null= 999519
index_where_null = num_rows - num_rows_null
all_data_df = all_data_df.drop(all_data_df.index[index_where_null:(index_where_null+num_rows_null)])
all_data_df
null_columns=all_data_df.columns[all_data_df.isnull().any()]
all_data_df[null_columns].isnull().sum()

Price            999519
Neighborhood     999519
Borough          999519
Latitude         999519
Longitude        999519
Review          1031751
Room Type        999527
dtype: int64


Review       32232
Room Type        8
dtype: int64

In [46]:
#Getting Date column to just show year#
rows = [];
for row in all_data_df.loc[:,"Date"]:
    row = row[0:4]
    rows.append(row)
all_data_df["Date"] = rows
len(all_data_df[all_data_df["Date"] == '2018'])

C:\Users\romys\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


49056

In [47]:
#write all data to csv#
all_data_df.to_csv("output_csv/all_historical_data.csv")

In [48]:
#Create average price dataframe#
pd.to_numeric(all_data_df["Price"])
price_summary_df = pd.DataFrame(all_data_df.groupby("Date")["Price"].mean())
price_summary_df = price_summary_df.rename(index=str,columns={"Price":"Average Price"})
price_summary_df

,Average Price
Date,
2014,211.037473
2015,157.174674
2016,147.088838
2017,136.103528
2018,152.330031


In [49]:
#write price summary data to csv#
price_summary_df.to_csv("output_csv/historical_price_data.csv")

In [50]:
#Create borough average price summary dataframe#
neighborhood_summary_df = pd.DataFrame(all_data_df.groupby(["Borough", "Neighborhood","Date"])["Price"].mean())
neighborhood_summary_df.reset_index(inplace=True)
neighborhood_summary_df
neighborhood_summary_df['Neighborhood'].str.contains('Allerton').any()
choro_df = pd.DataFrame(neighborhood_summary_df.groupby("Neighborhood")["Price"].mean()).reset_index()
choro_df
data_df = neighborhood_summary_df[neighborhood_summary_df.loc[:,"Neighborhood"]=="Allerton"][["Date", "Price"]]
for index, row in data_df.iterrows():
    print(row['Date'], row['Price'])

2014 72.35714285714286
2015 63.13333333333333
2016 62.76923076923077
2017 67.0
2018 89.28947368421052


In [51]:
#write summary data to csv#
neighborhood_summary_df.to_csv("output_csv/neighborhood_data.csv")

In [52]:
borough_summary_df = pd.DataFrame(neighborhood_summary_df.groupby(["Borough", "Date"])["Price"].mean())
borough_summary_df

Price
Borough       Date            
Bronx         2014   95.059080
              2015   78.424907
              2016   82.224539
              2017   97.548146
              2018   87.109599
Brooklyn      2014  152.155500
              2015  122.756668
              2016  125.952936
              2017  125.762991
              2018  131.196663
Manhattan     2014  266.421077
              2015  196.447977
              2016  197.434403
              2017  179.722155
              2018  208.564560
Queens        2014  117.212531
              2015  101.994372
              2016  112.868443
              2017  102.533153
              2018  114.033138
Staten Island 2014  370.243754
              2015  184.963548
              2016  140.436670
              2017  128.118728
              2018  131.188337

In [53]:
#write sumary data to csv#
borough_summary_df.to_csv("output_csv/borough_historical_data.csv")

In [54]:
#Create room_type average price summary dataframe#
neighborhood_roomtype_summary_df = pd.DataFrame(all_data_df.groupby(["Borough","Room Type","Date"])["Price"].mean())
neighborhood_roomtype_summary_df

Price
Borough       Room Type       Date            
Bronx         Entire home/apt 2014  135.907895
                              2015  133.181034
                              2016  127.082873
                              2017  127.729469
                              2018  117.576433
              Private room    2014   75.643836
                              2015   65.940075
                              2016   67.445813
                              2017   57.911523
                              2018   67.960139
              Shared room     2014   68.500000
                              2015   52.372093
                              2016   56.187500
                              2017   56.291667
                              2018   80.297297
Brooklyn      Entire home/apt 2014  214.703974
                              2015  178.317598
                              2016  176.627831
                              2017  164.364030
                              2018  175.497537
              Private room    2014  100.429926
                              2015   78.097930
                              2016   77.949925
                              2017   70.060560
                              2018   74.969941
              Shared room     2014   71.927336
                              2015   56.386503
                              2016   56.749478
                              2017   52.786932
                              2018   53.670157
...                                        ...
Queens        Entire home/apt 2014  168.819672
                              2015  141.207305
                              2016  141.156274
                              2017  136.959615
                              2018  145.179173
              Private room    2014   88.129630
                              2015   70.038592
                              2016   69.225914
                              2017   65.303634
                              2018   76.156270
              Shared room     2014  106.682353
                              2015   78.544715
                              2016   52.961905
                              2017   49.528090
                              2018   65.804878
Staten Island Entire home/apt 2014  374.061538
                              2015  266.647887
                              2016  175.768293
                              2017  190.183333
                              2018  174.538462
              Private room    2014   72.886792
                              2015   69.513043
                              2016   69.547619
                              2017   65.132075
                              2018   59.779070
              Shared room     2014   41.000000
                              2015   45.000000
                              2016   31.500000
                              2017   14.000000
                              2018   52.600000

[75 rows x 1 columns]

In [55]:
#write room type data to csv#
neighborhood_roomtype_summary_df.to_csv("output_csv/roomtype_historical_data.csv")

In [56]:
import json

with open('neighborhoods.geojson') as f:
    data = json.load(f)

count_add = 0
count_del = 0
indices = [];
count = 0

print(len(data['features']))

for feature in data['features']:
    change_property = feature['properties']
    neigh_name = feature['properties']['neighbourhood']
    data_df = neighborhood_summary_df[neighborhood_summary_df.loc[:,"Neighborhood"]== neigh_name][["Date", "Price"]]

    if neighborhood_summary_df["Neighborhood"].str.contains(neigh_name).any() and (len(data_df) == 5):
        count_add = count_add + 1
        
        for index, row in data_df.iterrows():
            change_property[row['Date']] = round(row['Price'],2)
        
    else:
        count_del = count_del + 1
        indices.append(count)
        #data["features"].remove(feature)
    count = count + 1

print(len(data['features']))  
print(count_add)
print(count_del)
print(count)
print(indices)
print(len(indices))

count = 0
for ind in indices:
    data["features"].remove(data["features"][ind-count])
    count = count + 1

print(len(data['features'])) 
data["features"][0]["properties"]

233
233
185
48
233
[5, 9, 10, 11, 14, 19, 20, 25, 28, 31, 39, 42, 47, 49, 66, 67, 72, 87, 100, 104, 106, 108, 111, 112, 115, 120, 121, 124, 125, 129, 135, 136, 147, 153, 157, 159, 161, 170, 173, 179, 181, 187, 191, 192, 194, 224, 228, 232]
48
185


{'neighbourhood': 'Bayswater',
 'neighbourhood_group': 'Queens',
 '2014': 103.33,
 '2015': 66.86,
 '2016': 80.62,
 '2017': 70.67,
 '2018': 74.11}

In [57]:
with open('neighborhoods_price.geojson', 'w') as outfile:
     geojson.dump(data, outfile)

In [58]:
len(choro_df.loc[:,"Neighborhood"])

246

In [123]:
name_list = []
length = len(response["features"])
for i in range(0,length):
    change = response["features"][i]["properties"]
    name_list.append(change["NTAName"])
    if choro_df["Neighborhood"].str.contains(change["NTAName"]).any() & (not choro_df[choro_df.loc[:,"Neighborhood"]==change["NTAName"]]["Price"].empty) :
        change["Average Price"] = choro_df[choro_df.loc[:,"Neighborhood"]==change["NTAName"]]["Price"].values[0]
print(sorted(name_list))

['Airport', 'Allerton-Pelham Gardens', "Annadale-Huguenot-Prince's Bay-Eltingville", 'Arden Heights', 'Astoria', 'Auburndale', 'Baisley Park', 'Bath Beach', 'Battery Park City-Lower Manhattan', 'Bay Ridge', 'Bayside-Bayside Hills', 'Bedford', 'Bedford Park-Fordham North', 'Bellerose', 'Belmont', 'Bensonhurst East', 'Bensonhurst West', 'Borough Park', 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel', 'Briarwood-Jamaica Hills', 'Brighton Beach', 'Bronxdale', 'Brooklyn Heights-Cobble Hill', 'Brownsville', 'Bushwick North', 'Bushwick South', 'Cambria Heights', 'Canarsie', 'Carroll Gardens-Columbia Street-Red Hook', 'Central Harlem North-Polo Grounds', 'Central Harlem South', 'Charleston-Richmond Valley-Tottenville', 'Chinatown', 'Claremont-Bathgate', 'Clinton', 'Clinton Hill', 'Co-op City', 'College Point', 'Corona', 'Crotona Park East', 'Crown Heights North', 'Crown Heights South', 'Cypress Hills-City Line', 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill', 'Douglas Manor-Douglas

C:\Users\romys\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [118]:
print(length)
len(index_del)


195


126